In [ ]:
#default_exp decorator

# fastcgi.decorator
> Instant fastcgi scripts by just adding a decorator

In [ ]:
#export
from fastcgi.core import FcgiHandler,TextWrapper
from fastcore.basics import *
from fastcore.imports import *
from socketserver import UnixStreamServer,TCPServer,ForkingMixIn,ForkingTCPServer
import inspect,os,sys

In [ ]:
#export
_all_ = ['os','sys']

In [ ]:
from fastcore.utils import *
from fastcore.script import *
import subprocess,time,urllib

In [ ]:
#export
class ForkingUnixServer(ForkingMixIn, UnixStreamServer): pass

In [ ]:
#export
def fastcgi(sock='fcgi.sock', func=None):
    if callable(sock): sock,func = 'fcgi.sock',sock
    if isinstance(sock,Path): sock = str(sock)
    if func is None: return partial(fastcgi, sock)

    mod = inspect.getmodule(inspect.currentframe().f_back)
    class DecorateHandler(FcgiHandler):
        def handle(self):
            oldin,oldout,oldenv = sys.stdin,sys.stdout,os.environ
            try:
                sys.stdin,sys.stdout,os.environ = TextWrapper(self['stdin']),TextWrapper(self['stdout']),self.environ
                func()
            finally: sys.stdin,sys.stdout,os.environ = oldin,oldout,oldenv

    srv_type = ForkingUnixServer if isinstance(sock,str) else ForkingTCPServer
    f = partial(srv_type, sock, DecorateHandler)
    if mod and mod.__name__=="__main__":
        try:
            with f() as srv: srv.serve_forever()
        except KeyboardInterrupt:
            if isinstance(sock,str):os.unlink(sock)
    else: return f

In [ ]:
def c():
    return "Content-type: text/html\r\n\r\n<html>" \
        + f'{sys.stdin.read()} ; {os.environ["QUERY_STRING"]}</html>\r\n'

In [ ]:
p = Path('fcgi.sock')

@fastcgi(p)
def hello(): sys.stdout.write(c())

@threaded
def _f():
    with hello() as srv: srv.handle_request()

In [ ]:
proc = subprocess.Popen(['./http2fcgi'])
if p.exists(): p.unlink()
t = _f()
time.sleep(0.2)
res = urlread('http://localhost:6065/setup.py?a=1', foo='bar', greeting='你好')
print(urllib.parse.unquote(res.decode()))
proc.terminate()

<html>foo=bar&greeting=你好 ; a=1</html>



In [ ]:
addr = 'localhost',8003

@fastcgi(addr)
def hello(): sys.stdout.write(c())

In [ ]:
time.sleep(0.5)
proc = subprocess.Popen('./http2fcgi -fcgi tcp://localhost:8003'.split())
t = _f()
time.sleep(0.2)
res = urlread('http://localhost:6065/setup.py?a=1', foo='bar', greeting='你好')
print(urllib.parse.unquote(res.decode()))
proc.terminate()

## fin -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_decorator.ipynb.
Converted 02_http.ipynb.
Converted index.ipynb.
